In [17]:
import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
%matplotlib inline

In [21]:
# Define a transform to normalize the data
data_transform = transforms.ToTensor()

# Download and load the training data
trainset = datasets.FashionMNIST('F_MNIST_data/', download=True, train=True, transform=data_transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)

# Download and load the test data
testset = datasets.FashionMNIST('F_MNIST_data/', download=True, train=False, transform=data_transform)
testloader = DataLoader(testset, batch_size=64, shuffle=True)

In [24]:
print('Train data, number of images: ', len(trainset))
print('Test data, number of images: ', len(testset))

Train data, number of images:  60000
Test data, number of images:  10000


In [7]:
input_size = 784
hidden_sizes = [128, 64]
output_size = 10

model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[1], output_size))

model

Sequential(
  (0): Linear(in_features=784, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=64, bias=True)
  (3): ReLU()
  (4): Linear(in_features=64, out_features=10, bias=True)
)

In [25]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [27]:
# Implement a function for the validation pass
def validation(model, testloader, criterion):
    test_loss = 0
    accuracy = 0
    for images, labels in testloader:

        images.resize_(images.shape[0], 784)

        output = model.forward(images)
        test_loss += criterion(output, labels).item()

        ps = torch.exp(output)
        equality = (labels.data == ps.max(dim=1)[1])
        accuracy += equality.type(torch.FloatTensor).mean()
    
    return test_loss, accuracy

In [29]:
epochs = 3
steps = 0
running_loss = 0
print_every = 40

for e in range(epochs):
    model.train()
    for images, labels in iter(trainloader):
        
        steps += 1
        images.resize_(images.size()[0], 784)
        optimizer.zero_grad()
        
        output = model.forward(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if steps % print_every == 0:
            # Make sure network is in eval mode for inference
            model.eval()
            
            # Turn off gradients for validation, saves memory and computations
            with torch.no_grad():
                test_loss, accuracy = validation(model, testloader, criterion)
                
            print("Epoch: {}/{}.. ".format(e+1, epochs),
                  "Training Loss: {:.3f}.. ".format(running_loss/print_every),
                  "Test Loss: {:.3f}.. ".format(test_loss/len(testloader)),
                  "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))
            
            running_loss = 0
            
            # Make sure training is back on
            model.train()

Epoch: 1/3..  Training Loss: 0.557..  Test Loss: 0.585..  Test Accuracy: 0.797
Epoch: 1/3..  Training Loss: 0.578..  Test Loss: 0.598..  Test Accuracy: 0.791
Epoch: 1/3..  Training Loss: 0.591..  Test Loss: 0.578..  Test Accuracy: 0.796
Epoch: 1/3..  Training Loss: 0.552..  Test Loss: 0.598..  Test Accuracy: 0.790
Epoch: 1/3..  Training Loss: 0.548..  Test Loss: 0.605..  Test Accuracy: 0.787
Epoch: 1/3..  Training Loss: 0.544..  Test Loss: 0.571..  Test Accuracy: 0.802
Epoch: 1/3..  Training Loss: 0.541..  Test Loss: 0.573..  Test Accuracy: 0.800
Epoch: 1/3..  Training Loss: 0.549..  Test Loss: 0.566..  Test Accuracy: 0.804
Epoch: 1/3..  Training Loss: 0.554..  Test Loss: 0.569..  Test Accuracy: 0.805
Epoch: 1/3..  Training Loss: 0.544..  Test Loss: 0.578..  Test Accuracy: 0.796
Epoch: 1/3..  Training Loss: 0.558..  Test Loss: 0.565..  Test Accuracy: 0.803
Epoch: 1/3..  Training Loss: 0.539..  Test Loss: 0.560..  Test Accuracy: 0.807
Epoch: 1/3..  Training Loss: 0.531..  Test Loss: 0.5

In [33]:
model.eval()

dataiter = iter(testloader)
images, labels = dataiter.next()
img = images[0]
# Convert 2D image to 1D vector
img = img.view(1, 784)

# Calculate the class probabilities (softmax) for img
with torch.no_grad():
    output = model.forward(img)

ps = torch.exp(output)

# Plot the image and probabilities


tensor([[7.4575e+00, 1.3439e+01, 2.0471e+00, 4.4669e+02, 1.5529e+00, 1.5067e-02,
         8.1384e-01, 4.0921e-02, 2.4455e-01, 9.1974e-02]])

In [36]:
labels[0]

tensor(3)